**Assignment 2 - ngram models**

In [7]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
df_validation = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
finnish_df = df[df['lang'] == 'fi']['question']
japanese_df = df[df['lang'] == 'ja']['question']
russian_df = df[df['lang'] == 'ru']['question']
english_df = df['context']

In [9]:
finnish_validation_df = df_validation[df_validation['lang'] == 'fi']['question']
japanese_validation_df = df_validation[df_validation['lang'] == 'ja']['question']
russian_validation_df = df_validation[df_validation['lang'] == 'ru']['question']
english_validation_df = df_validation['context']


In [10]:
english_validation_df

,context
0,Portland is the largest city in the U.S. state...
1,"The word cholera is from ""kholera"" from χολή ""..."
2,Since it became widespread in the 19th century...
3,World War I occurred from 1914 to 1918. In ter...
4,"World War I (often abbreviated as WWI or WW1),..."
...,...
3023,Reyyalagadda is a village belonging to Gangara...
3024,Boothumillipadu is a village in Gannavaram man...
3025,Mallavemula is a village belonging to Chagalam...
3026,"Andria Urushadze (; born April 25, 1968) is a ..."


In [5]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
!pip install nltk


Attempt with NLTK

In [2]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import Laplace, KneserNeyInterpolated, WittenBellInterpolated, Lidstone, MLE
from nltk.tokenize import word_tokenize, sent_tokenize
from functools import partial
from nltk.util import everygrams, flatten
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [4]:
def calculate_perplexity_Laplace(train_data, test_data, n=5):
    # Tokenize the data
    train_data_tokenized = [list(map(str.lower, word_tokenize(sent))) for sent in train_data]
    test_data_tokenized = [list(map(str.lower, word_tokenize(sent))) for sent in test_data]

    # Prepare the data for the language model
    train_data, padded_vocab = padded_everygram_pipeline(n, train_data_tokenized)

    # Train the model with Laplace smoothing
    model = Laplace(n)
    model.fit(train_data, padded_vocab)

    # Prepare the test data for perplexity calculation
    test_data_ngrams, _ = padded_everygram_pipeline(n, test_data_tokenized)

    # Convert the test_data generator to a list of n-grams
    test_data_ngrams_list = [list(ngrams) for ngrams in test_data_ngrams]

    # Flatten the list of lists into a single list of n-grams
    flattened_test_data_ngrams = [ngram for sublist in test_data_ngrams_list for ngram in sublist]

    # Calculate perplexity
    perplexity = model.perplexity(flattened_test_data_ngrams)

    # Get the size of the vocabulary
    vocab_size = len(model.vocab)

    return perplexity, vocab_size, model

In [5]:
#edit to n=5 for the second model
ngramsize=3

In [12]:
# Calculate perplexity for each dataset
finnish_perplexity, finnish_vocab_size, finnish_model = calculate_perplexity_Laplace(finnish_df, finnish_df, n=ngramsize)
print(f'Finnish Perplexity: {finnish_perplexity}, Vocabulary Size: {finnish_vocab_size}')

japanese_perplexity, japanese_vocab_size, japanese_model = calculate_perplexity_Laplace(japanese_df, japanese_df, n=ngramsize)
print(f'Japanese Perplexity: {japanese_perplexity}, Vocabulary Size: {japanese_vocab_size}')

russian_perplexity, russian_vocab_size, russian_model = calculate_perplexity_Laplace(russian_df, russian_df, n=ngramsize)
print(f'Russian Perplexity: {russian_perplexity}, Vocabulary Size: {russian_vocab_size}')

english_perplexity, english_vocab_size, english_model = calculate_perplexity_Laplace(english_df, english_df, n=ngramsize)
print(f'English Perplexity: {english_perplexity}, Vocabulary Size: {english_vocab_size}')


Finnish Perplexity: 179.35638578310903, Vocabulary Size: 4421
Japanese Perplexity: 81.71054095888772, Vocabulary Size: 2400
Russian Perplexity: 275.1710778106379, Vocabulary Size: 4867
English Perplexity: 4445.265987464632, Vocabulary Size: 83494


Experimentation

In [13]:
generate_sent(english_model, num_words=50, random_seed=69)

"of agreements are, however neither of which east godavari district or paschima godavari jilla is one of patrick brontë's death in 80% of carbon dioxide in the united states air force), and doubts as to the public on november 22, 1876 (at right"

In [14]:
english_model.score('What')

5.2235851136182e-07

KneserNeyInterpolated

In [15]:
def calculate_perplexity_WrittenBell(train_data, test_data, n=5):
    # Tokenize the data
    train_data_tokenized = [list(map(str.lower, word_tokenize(sent))) for sent in train_data]
    test_data_tokenized = [list(map(str.lower, word_tokenize(sent))) for sent in test_data]

    # Prepare the data for the language model
    train_data, padded_vocab = padded_everygram_pipeline(n, train_data_tokenized)

    # Train the model with Laplace smoothing
    model = WittenBellInterpolated(n)
    model.fit(train_data, padded_vocab)

    # Prepare the test data for perplexity calculation
    test_data_ngrams, _ = padded_everygram_pipeline(n, test_data_tokenized)

    # Convert the test_data generator to a list of n-grams
    test_data_ngrams_list = [list(ngrams) for ngrams in test_data_ngrams]

    # Flatten the list of lists into a single list of n-grams
    flattened_test_data_ngrams = [ngram for sublist in test_data_ngrams_list for ngram in sublist]

    # Calculate perplexity
    perplexity = model.perplexity(flattened_test_data_ngrams)

    # Get the size of the vocabulary
    vocab_size = len(model.vocab)

    return perplexity, vocab_size, model

In [ ]:
# Calculate perplexity for each dataset
finnish_perplexity, finnish_vocab_size, finnish_model = calculate_perplexity_WrittenBell(finnish_df, finnish_df, n=ngramsize)
print(f'Finnish Perplexity: {finnish_perplexity}, Vocabulary Size: {finnish_vocab_size}')

japanese_perplexity, japanese_vocab_size, japanese_model = calculate_perplexity_WrittenBell(japanese_df, japanese_df, n=ngramsize)
print(f'Japanese Perplexity: {japanese_perplexity}, Vocabulary Size: {japanese_vocab_size}')

russian_perplexity, russian_vocab_size, russian_model = calculate_perplexity_WrittenBell(russian_df, russian_df, n=ngramsize)
print(f'Russian Perplexity: {russian_perplexity}, Vocabulary Size: {russian_vocab_size}')

english_perplexity, english_vocab_size, english_model = calculate_perplexity_WrittenBell(english_df, english_df, n=ngramsize)
print(f'English Perplexity: {english_perplexity}, Vocabulary Size: {english_vocab_size}')


Finnish Perplexity: 13.98932372188909, Vocabulary Size: 4421
Japanese Perplexity: 13.469293755464399, Vocabulary Size: 2400
Russian Perplexity: 16.73508743543924, Vocabulary Size: 4867
